In [1]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
from pyspark.sql import SparkSession

In [3]:
# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("MongoDB to Spark") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/dbmycrawler.tblcafeland") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/dbmycrawler.tblcafeland") \
    .getOrCreate()

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.UnsupportedOperationException: getSubject is supported only if a security manager is allowed
	at java.base/javax.security.auth.Subject.getSubject(Subject.java:347)
	at org.apache.hadoop.security.UserGroupInformation.getCurrentUser(UserGroupInformation.java:577)
	at org.apache.spark.util.Utils$.$anonfun$getCurrentUserName$1(Utils.scala:2416)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.util.Utils$.getCurrentUserName(Utils.scala:2416)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:329)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:501)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.ja

In [ ]:
# Kết nối tới MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["dbmycrawler"] 
collection = db["tblcafeland"]  

In [ ]:
# 2. Đọc dữ liệu trực tiếp từ MongoDB
df = spark.read \
    .format("mongo") \
    .option("uri", "mongodb://localhost:27017/dbmycrawler.tblcafeland") \
    .load()

# 3. Hiển thị cấu trúc dữ liệu
df.printSchema()

In [4]:
from pyspark.sql.functions import regexp_replace, when, col
from pyspark.sql.types import DoubleType, DateType

In [212]:
#1. Xử lý đồng bộ đơn vị thành m2 cột diện tích
#Xóa bỏ dấu chấm ở các dòng m2
df = df.withColumn("Diện tích:", regexp_replace(col("Diện tích:"), "\\.", ""))

#Đổi dấu phẩy ở dòng có chứa 'ha' thành dấu chấm
df = df.withColumn("Diện tích:", regexp_replace(col("Diện tích:"), ",", "."))

#xóa bỏ chữ h2 và m2
df = df.withColumn("Diện tích:", regexp_replace(col("Diện tích:"), "ha|m2", ""))

#Dòng nào có chứa dấu . thì nhân với 10000 để dồng bộ đơn vị m2
df = df.withColumn("Diện tích:", 
                   when(col("Diện tích:").contains("."), col("Diện tích:").cast("double") * 10000) #Nếu chứa dấu . thì đổi sang kiểu double và * 10000
                   .otherwise(col("Diện tích:").cast("double"))) #Nếu không chứa dấu chấm thì chỉ chuyển sang kiểu double

df.show()

+------------------+--------------------+--------------------+--------------------+--------------------+-----------+----------+--------------------+-----------------+--------+--------------------+
|        Loại dự án|           Tên dự án|               Đường|           Thành phố|          Chủ đầu tư| Trạng thái|Diện tích:|           Ngày đăng|  Tổng vốn đầu tư|Xếp hạng|               Mô tả|
+------------------+--------------------+--------------------+--------------------+--------------------+-----------+----------+--------------------+-----------------+--------+--------------------+
|     Đất Nền Dự Án|The Long Eyes: Dự...|   đường Sông Thao 5|           Trảng Bom|Công Ty Cổ Phần B...|Đang mở bán|    8200.0|Cập nhật: 12/09/2...|    chưa cập nhật|     5.0|The Long Eyes là ...|
|Nhà Phố - Biệt Thự|Tổ hợp biệt thự, ...|khu đô thị Ciputr...|              Tây Hồ|   Tập đoàn Sunshine|Đang mở bán|  101000.0|Cập nhật: 27/09/2...|    chưa cập nhật|     0.0|Noble Palace Tây ...|
|    Loại Hình 

In [213]:
# 2. Làm sạch cột 'Tổng vốn đầu tư'
# Loại bỏ "tỷ đồng" và chuyển đổi sang số thực
df = df.withColumn("Tổng vốn đầu tư", 
                   when(col("Tổng vốn đầu tư").like("%tỷ đồng%"), 
                        regexp_replace(col("Tổng vốn đầu tư"), " tỷ đồng", "").cast(DoubleType()) * 1000)  # chuyển tỷ đồng thành số
                   .otherwise("unknow")
                   )  # cho các giá trị chưa cập nhật thành unknow

df.show()

+------------------+--------------------+--------------------+--------------------+--------------------+-----------+----------+--------------------+---------------+--------+--------------------+
|        Loại dự án|           Tên dự án|               Đường|           Thành phố|          Chủ đầu tư| Trạng thái|Diện tích:|           Ngày đăng|Tổng vốn đầu tư|Xếp hạng|               Mô tả|
+------------------+--------------------+--------------------+--------------------+--------------------+-----------+----------+--------------------+---------------+--------+--------------------+
|     Đất Nền Dự Án|The Long Eyes: Dự...|   đường Sông Thao 5|           Trảng Bom|Công Ty Cổ Phần B...|Đang mở bán|    8200.0|Cập nhật: 12/09/2...|         unknow|     5.0|The Long Eyes là ...|
|Nhà Phố - Biệt Thự|Tổ hợp biệt thự, ...|khu đô thị Ciputr...|              Tây Hồ|   Tập đoàn Sunshine|Đang mở bán|  101000.0|Cập nhật: 27/09/2...|         unknow|     0.0|Noble Palace Tây ...|
|    Loại Hình Khác|Noble

In [214]:
# 3. Làm sạch cột 'Ngày đăng'
# Loại bỏ "Cập nhật: " 
df = df.withColumn("Ngày đăng", 
                   regexp_replace(col("Ngày đăng"), "Cập nhật: ", "\\."))

df.show()

+------------------+--------------------+--------------------+--------------------+--------------------+-----------+----------+--------------------+---------------+--------+--------------------+
|        Loại dự án|           Tên dự án|               Đường|           Thành phố|          Chủ đầu tư| Trạng thái|Diện tích:|           Ngày đăng|Tổng vốn đầu tư|Xếp hạng|               Mô tả|
+------------------+--------------------+--------------------+--------------------+--------------------+-----------+----------+--------------------+---------------+--------+--------------------+
|     Đất Nền Dự Án|The Long Eyes: Dự...|   đường Sông Thao 5|           Trảng Bom|Công Ty Cổ Phần B...|Đang mở bán|    8200.0| .12/09/2024 8:49 AM|         unknow|     5.0|The Long Eyes là ...|
|Nhà Phố - Biệt Thự|Tổ hợp biệt thự, ...|khu đô thị Ciputr...|              Tây Hồ|   Tập đoàn Sunshine|Đang mở bán|  101000.0| .27/09/2024 6:13 PM|         unknow|     0.0|Noble Palace Tây ...|
|    Loại Hình Khác|Noble

In [216]:
# 4. Xử lý cột 'Trạng thái'
# Thay thế null bằng "chưa cập nhật"
df = df.withColumn("Trạng thái", 
                   when(col("Trạng thái") == None, "Chưa cập nhật").otherwise(col("Trạng thái")))
df.show()

+------------------+--------------------+--------------------+--------------------+--------------------+-----------+----------+--------------------+---------------+--------+--------------------+
|        Loại dự án|           Tên dự án|               Đường|           Thành phố|          Chủ đầu tư| Trạng thái|Diện tích:|           Ngày đăng|Tổng vốn đầu tư|Xếp hạng|               Mô tả|
+------------------+--------------------+--------------------+--------------------+--------------------+-----------+----------+--------------------+---------------+--------+--------------------+
|     Đất Nền Dự Án|The Long Eyes: Dự...|   đường Sông Thao 5|           Trảng Bom|Công Ty Cổ Phần B...|Đang mở bán|    8200.0| .12/09/2024 8:49 AM|         unknow|     5.0|The Long Eyes là ...|
|Nhà Phố - Biệt Thự|Tổ hợp biệt thự, ...|khu đô thị Ciputr...|              Tây Hồ|   Tập đoàn Sunshine|Đang mở bán|  101000.0| .27/09/2024 6:13 PM|         unknow|     0.0|Noble Palace Tây ...|
|    Loại Hình Khác|Noble

In [205]:
# Chuyển đổi DataFrame thành Spark DataFrame
spark_df = spark.createDataFrame(df)

In [ ]:
# Ghi dữ liệu vào collection mới hoặc collection hiện tại
spark_df.write \
    .format("mongo") \
    .mode("append") \
    .option("uri", "mongodb://localhost:27017/ten_cua_database.ten_cua_collection") \
    .save()

In [ ]:
# Ghi dữ liệu đã xử lý thành file CSV
output_path = "/path/to/output/clean_cafeland.csv"  # Thay đổi đường dẫn lưu file
spark_df.coalesce(1).write.csv(output_path, header=True, mode="overwrite")

print(f"Quá trình lưu dữ liệu thành file CSV tại: {output_path} đã hoàn tất.")